<a href="https://colab.research.google.com/github/Prisze/AAI-520-Final-Project/blob/Priscilla's-Model-Architecture%2FTraining/AAI_520_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
import numpy as np
import pandas as pd
import torch

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Datasets/UbuntuDataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Detect of GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Datasets/UbuntuDataset'):
    for filename in filenames:  # loops through all the files in the directories
        print(filename) # Gives a file name, without the complete file path
        filepath = os.path.join(dirname,filename) # Completes the file path
        filename = filename.split('.')[0] # Removes the file extension for naming
        exec(f"{filename} = pd.read_csv('{filepath}')")


toc.csv
dialogueText.csv
dialogueText_196.csv
dialogueText_301.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dialogueText.groupby('dialogueID')['from'].nunique().reset_index()['from'].value_counts()

,count
from,
2,339963
1,6145


In [ ]:
# Remove monologues
# Group consequtive messages of the same participant into one
  # Messages should be interleaved (Person A, Person B, Person A, Person B...)

In [ ]:
#Find dialogueIDs with only one unique 'from'
single_from_ids = dialogueText.groupby('dialogueID')['from'].nunique().reset_index()
single_from_ids = single_from_ids[single_from_ids['from'] == 1]['dialogueID']

#Filter out those dialogueIDs from the original DataFrame
filtered_dialogueText = dialogueText[~dialogueText['dialogueID'].isin(single_from_ids)]
filtered_dialogueText

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
...,...,...,...,...,...,...
1038319,3,51506.tsv,2012-01-31T10:56:00.000Z,DJ,NaN,anyone on?
1038320,3,51506.tsv,2012-01-31T10:56:00.000Z,aeon-ltd,DJ,yes
1038321,3,99669.tsv,2008-11-16T20:11:00.000Z,KR-data,NaN,can I get a pastebin of someones menu.lst with...
1038322,3,99669.tsv,2008-11-16T20:12:00.000Z,outbackwifi,KR-data,http://pastebin.com/fe921690


In [ ]:
filtered_dialogueText.loc[filtered_dialogueText['dialogueID'] == '64545.tsv']['text'].tolist()

['Sock Puppe?t',
 'WTF?',
 "it's a wikipedia term.  There is Prodigy, PRDIGY and prgidy and they're all the same guy"]

In [ ]:
# Create a helper column to identify groups of consecutive messages from the same participant
filtered_dialogueText['group'] = (filtered_dialogueText['from'] != filtered_dialogueText['from'].shift()).cumsum()

# Group by dialogueID and the helper column, and aggregate messages
grouped = filtered_dialogueText.groupby(['dialogueID', 'group']).agg({
    'from': 'first',  # Keep the participant's name
    'to': 'first',    # Keep the 'to' column (assuming it's consistent)
    'date': 'first',  # Keep the first date (or use another method if needed
    'text': lambda x: ' '.join(str(m) for m in x if pd.notna(m))  # Combine messages into one string, ignoring NaNs
}).reset_index()

# Display the result
print(grouped)

<ipython-input-34-a0b43486419a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dialogueText['group'] = (filtered_dialogueText['from'] != filtered_dialogueText['from'].shift()).cumsum()


       dialogueID   group      from        to                      date  \
0           1.tsv  339021      seth      None  2005-05-25T11:26:00.000Z   
1           1.tsv  339022  lifeless      seth  2005-05-25T11:27:00.000Z   
2           1.tsv  339023      seth  lifeless  2005-05-25T11:27:00.000Z   
3          10.tsv  105421      jdz`      None  2004-11-27T09:10:00.000Z   
4          10.tsv  105422  lifeless      jdz`  2004-11-27T09:12:00.000Z   
...           ...     ...       ...       ...                       ...   
770870  99998.tsv  379784   zcat[1]      None  2008-08-28T10:03:00.000Z   
770871  99998.tsv  379785      fab3   zcat[1]  2008-08-28T10:04:00.000Z   
770872  99998.tsv  379786   zcat[1]      fab3  2008-08-28T10:06:00.000Z   
770873  99999.tsv  115752    alex65      None  2008-09-02T23:01:00.000Z   
770874  99999.tsv  115753    zinake    alex65  2008-09-02T23:02:00.000Z   

                                                     text  
0       Also guys, I'm trying to get in

In [ ]:
#grouped.loc[grouped['dialogueID'] == '126125.tsv']['text'].tolist()

grouped

,dialogueID,group,from,to,date,text
0,1.tsv,339021,seth,None,2005-05-25T11:26:00.000Z,"Also guys, I'm trying to get into my FIrefox p..."
1,1.tsv,339022,lifeless,seth,2005-05-25T11:27:00.000Z,are you logged in as 'root' ?
2,1.tsv,339023,seth,lifeless,2005-05-25T11:27:00.000Z,no.
3,10.tsv,105421,jdz`,None,2004-11-27T09:10:00.000Z,ugh ;( http://planet.ubuntulinux.org seems to...
4,10.tsv,105422,lifeless,jdz`,2004-11-27T09:12:00.000Z,"perhaps if you define *broken* a little, we ca..."
...,...,...,...,...,...,...
770870,99998.tsv,379784,zcat[1],None,2008-08-28T10:03:00.000Z,umm !res is way out of date or something? dpkg...
770871,99998.tsv,379785,fab3,zcat[1],2008-08-28T10:04:00.000Z,its true
770872,99998.tsv,379786,zcat[1],fab3,2008-08-28T10:06:00.000Z,so i have an old nvidia card that was working ...
770873,99999.tsv,115752,alex65,None,2008-09-02T23:01:00.000Z,Does anyone know a good linux distribution ?


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2Model.from_pretrained('distilgpt2')

text = grouped['text']#"Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]